In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import  sklearn.linear_model as linearModel
from sklearn import preprocessing
from sklearn.manifold import TSNE
import numpy as np
#import seaborn
import matplotlib.pyplot as plt
from itertools import cycle
from warnings import filterwarnings
import timeit
import pymc3 as pm
import sys
import json
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression


In [2]:
sys.path.insert(0,'../src')
import bridgeSampling as bs # it contains a method to estimate the marginal likelihood according to the Bridge Sampling approach
import utilFunctions as ut          # it has different methods to handle and plot data
import BayesianModels as bm # it has methods to build and train bayesian model (Logistic Regression and Neural Nets)
import experiments as exp
import samplebiasselection as sbs

In [3]:
number = 3
fileTrain = '/home/francisco/Escritorio/code/vsbms/data/BIASEDFATS/Train_rrlyr-'+str(number)+'.csv'
fileTest = '/home/francisco/Escritorio/code/vsbms/data/BIASEDFATS/Test_rrlyr-'+str(number)+'.csv'

# Basic Ridge Logistic Regresion (Polynomial)

In [4]:
Cvalue = 1
plot = False
basicLR_stats_acc = []
basicLR_stats_f1 = []
res = []
for k in [1, 2, 3]:
    print('kernel '+str(k))
    for components in [2, 4, 6, 8, 10, 12, 14]:
        for size in[1000, 2000, 4000]:
            print('components '+str(components))
            dataTrain = pd.read_csv(fileTrain)
            dataTest = pd.read_csv(fileTest)
            try:
                dataTrain = dataTrain.loc[:, ~dataTrain.columns.str.contains('^Unnamed')]
                dataTrain =  dataTrain.drop(['Pred', 'Pred2', 'h', 'e', 'u','ID'], axis = 1)
                dataTrain = dataTrain.loc[:, (dataTrain != dataTrain.iloc[0]).any()]
                dataTest = dataTest[list(dataTrain.columns)]

            except:
                print('---')
            dataTrain = ut.downSampling(dataTrain)
            try:
                dataTrain = dataTrain.sample(size, random_state = 0)
            except:
                print('sample bigger that population')
            yTrain = 1*(dataTrain['label'] == 'ClassA')
            yTestHO = 1*(dataTest['label'] == 'ClassA')
            del dataTrain['label']
            del dataTest['label']

            names = dataTrain.columns
            scaler = preprocessing.StandardScaler()
            dataTrain = scaler.fit_transform(dataTrain)
            dataTrain = pd.DataFrame(dataTrain, columns=names)

            dataTest = scaler.fit_transform(dataTest)
            dataTest = pd.DataFrame(dataTest, columns=names)


            pca = PCA(n_components=components)
            pca.fit(dataTrain)
            dataTrain = pca.transform(dataTrain)
            dataTrain = pd.DataFrame(dataTrain)
            dataTrain = ut.Polinomial(dataTrain,k)


            pca.fit(dataTest)
            dataTest = pca.transform(dataTest)
            dataTest = pd.DataFrame(dataTest)
            dataTest = ut.Polinomial(dataTest,k)
            #size = dataTrain.shape[0]

            start_1 = timeit.default_timer()
            skf = StratifiedKFold(n_splits=int(10))
            skf.get_n_splits(dataTrain, yTrain)
            acc_kfold_Train = []
            f1_kfold_Train = []
            i = 0

            for train_index, test_index in skf.split(dataTrain, yTrain):
                        i = i+1
                        print('training fold ' + str(i))
                        X_train, X_test = dataTrain.iloc[train_index,:], dataTrain.iloc[test_index,:]
                        y_train, y_test = yTrain.iloc[train_index], yTrain.iloc[test_index]

                        basicLR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial',
                                                     C=Cvalue).fit(X_train, y_train)
                        predictions_1_Train = basicLR.predict(X_test)

                        accTrain = accuracy_score(y_test, predictions_1_Train, normalize=True)
                        f1Train = f1_score(y_test, predictions_1_Train, pos_label = 1)

                        if plot:
                            cm = confusion_matrix(y_test, predictions_1_Train)
                            ut.plot_confusion_matrix(cm, ['all', 'rrlyr'], type = 'train')
                        print('Accuracy train: ', accTrain)
                        #print('Accuracy f1 Train: ', f1Train)
                        basicLR_stats_acc.append(accTrain)
                        basicLR_stats_f1.append(f1Train)
            accTrain = np.mean(basicLR_stats_acc)
            f1Train =np.mean(basicLR_stats_f1)
            print('----------------------------------------------------------')
            print('Summary Training')
            print('Mean Accuracy train: ', accTrain)
            print('Mean f1 Train: ', f1Train)
            stop_1 = timeit.default_timer()
            time_CV = stop_1 - start_1
            basicLR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial', C=Cvalue).fit(dataTrain, yTrain)
            predictions_1_Test = basicLR.predict(dataTest)
            print(predictions_1_Test.shape)
            print(yTestHO.shape)
            print('----------------------------------------------------------')
            print('Summary Testing')
            accTest = accuracy_score(yTestHO, predictions_1_Test, normalize=True)
            f1Test = f1_score(yTestHO, predictions_1_Test,pos_label = 1)
            if plot: 
                cm = confusion_matrix(yTestHO, predictions_1_Test)
                ut.plot_confusion_matrix(cm, ['all', 'rrlyr'], type = 'train')

            print(' Accuracy test: ', accTest)
            print(' f1 test: ', f1Test)
            print('----------------------------------------------------------')

            res.append([k, components, size,  accTrain, accTest, f1Train, f1Test])
            pd.DataFrame(res).to_csv(str(number)+'-BasicCV0107C'+str(Cvalue)+'.csv')

kernel 1
components 2
71
training fold 1
Accuracy train:  0.9
training fold 2
Accuracy train:  0.92
training fold 3
Accuracy train:  0.89
training fold 4
Accuracy train:  0.86
training fold 5
Accuracy train:  0.87
training fold 6
Accuracy train:  0.86
training fold 7
Accuracy train:  0.88
training fold 8
Accuracy train:  0.89
training fold 9
Accuracy train:  0.85
training fold 10
Accuracy train:  0.85
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.877
Mean f1 Train:  0.8782998697207441
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.6806587287482969
 f1 test:  0.38340119870064515
----------------------------------------------------------
components 2
71
training fold 1
Accuracy train:  0.915
training fold 2
Accuracy train:  0.88
training fold 3
Accuracy train:  0.87
training fold 4
Accuracy train:  0.905
training fold 5
Accuracy train:  0.84
training fold 6
Accuracy trai

71
training fold 1
Accuracy train:  0.955
training fold 2
Accuracy train:  0.9475
training fold 3
Accuracy train:  0.945
training fold 4
Accuracy train:  0.975
training fold 5
Accuracy train:  0.945
training fold 6
Accuracy train:  0.9325
training fold 7
Accuracy train:  0.935
training fold 8
Accuracy train:  0.9275
training fold 9
Accuracy train:  0.9675
training fold 10
Accuracy train:  0.94
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9267083333333334
Mean f1 Train:  0.928497972328452
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7989574077365085
 f1 test:  0.5056372906045156
----------------------------------------------------------
components 10
71
training fold 1
Accuracy train:  0.93
training fold 2
Accuracy train:  0.97
training fold 3
Accuracy train:  0.94
training fold 4
Accuracy train:  0.95
training fold 5
Accuracy train:  0.9
training fold 6
Accuracy tra

71
training fold 1
Accuracy train:  0.94
training fold 2
Accuracy train:  0.905
training fold 3
Accuracy train:  0.905
training fold 4
Accuracy train:  0.92
training fold 5
Accuracy train:  0.865
training fold 6
Accuracy train:  0.895
training fold 7
Accuracy train:  0.925
training fold 8
Accuracy train:  0.91
training fold 9
Accuracy train:  0.88
training fold 10
Accuracy train:  0.91
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9344891304347824
Mean f1 Train:  0.9364641147890652
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.6821396836680291
 f1 test:  0.3919222139117427
----------------------------------------------------------
components 2
71
training fold 1
Accuracy train:  0.9175
training fold 2
Accuracy train:  0.9125
training fold 3
Accuracy train:  0.885
training fold 4
Accuracy train:  0.92
training fold 5
Accuracy train:  0.895
training fold 6
Accuracy trai

71
training fold 1
Accuracy train:  0.93
training fold 2
Accuracy train:  0.98
training fold 3
Accuracy train:  0.97
training fold 4
Accuracy train:  0.97
training fold 5
Accuracy train:  0.95
training fold 6
Accuracy train:  0.98
training fold 7
Accuracy train:  0.96
training fold 8
Accuracy train:  0.96
training fold 9
Accuracy train:  0.95
training fold 10
Accuracy train:  0.95
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9405220588235295
Mean f1 Train:  0.9423803329182157
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7074699366151295
 f1 test:  0.40861296735407526
----------------------------------------------------------
components 10
71
training fold 1
Accuracy train:  0.955
training fold 2
Accuracy train:  0.965
training fold 3
Accuracy train:  0.96
training fold 4
Accuracy train:  0.97
training fold 5
Accuracy train:  0.935
training fold 6
Accuracy train:  0.

71
training fold 1
Accuracy train:  0.925
training fold 2
Accuracy train:  0.905
training fold 3
Accuracy train:  0.8975
training fold 4
Accuracy train:  0.915
training fold 5
Accuracy train:  0.9
training fold 6
Accuracy train:  0.9075
training fold 7
Accuracy train:  0.8875
training fold 8
Accuracy train:  0.9075
training fold 9
Accuracy train:  0.9175
training fold 10
Accuracy train:  0.8875
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9419444444444445
Mean f1 Train:  0.9438962764768568
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.6982998637521474
 f1 test:  0.4047869480868569
----------------------------------------------------------
components 4
71
training fold 1
Accuracy train:  0.94
training fold 2
Accuracy train:  0.98
training fold 3
Accuracy train:  0.97
training fold 4
Accuracy train:  0.98
training fold 5
Accuracy train:  0.93
training fold 6
Accuracy t

(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7337480007108583
 f1 test:  0.43442808607021516
----------------------------------------------------------
components 10
71
training fold 1
Accuracy train:  0.96
training fold 2
Accuracy train:  0.965
training fold 3
Accuracy train:  0.955
training fold 4
Accuracy train:  0.98
training fold 5
Accuracy train:  0.93
training fold 6
Accuracy train:  0.96
training fold 7
Accuracy train:  0.975
training fold 8
Accuracy train:  0.98
training fold 9
Accuracy train:  0.95
training fold 10
Accuracy train:  0.965
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9452633928571428
Mean f1 Train:  0.947132047795219
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.6747941472661572
 f1 test:  0.38028582394509286
----------------------------------------------------------
component

In [5]:
Cvalue = 100
plot = False
basicLR_stats_acc = []
basicLR_stats_f1 = []
res = []
for k in [1, 2, 3]:
    print('kernel '+str(k))
    for components in [2, 4, 6, 8, 10, 12]:
        for size in[1000, 2000, 4000]:
            print('components '+str(components))
            dataTrain = pd.read_csv(fileTrain)
            dataTest = pd.read_csv(fileTest)
            try:
                dataTrain = dataTrain.loc[:, ~dataTrain.columns.str.contains('^Unnamed')]
                dataTrain =  dataTrain.drop(['Pred', 'Pred2', 'h', 'e', 'u','ID'], axis = 1)
                dataTrain = dataTrain.loc[:, (dataTrain != dataTrain.iloc[0]).any()]
                dataTest = dataTest[list(dataTrain.columns)]

            except:
                print('---')
            dataTrain = ut.downSampling(dataTrain)
            try:
                dataTrain = dataTrain.sample(size, random_state = 0)
            except:
                print('sample bigger that population')
            yTrain = 1*(dataTrain['label'] == 'ClassA')
            yTestHO = 1*(dataTest['label'] == 'ClassA')
            del dataTrain['label']
            del dataTest['label']

            names = dataTrain.columns
            scaler = preprocessing.StandardScaler()
            dataTrain = scaler.fit_transform(dataTrain)
            dataTrain = pd.DataFrame(dataTrain, columns=names)

            dataTest = scaler.fit_transform(dataTest)
            dataTest = pd.DataFrame(dataTest, columns=names)


            pca = PCA(n_components=components)
            pca.fit(dataTrain)
            dataTrain = pca.transform(dataTrain)
            dataTrain = pd.DataFrame(dataTrain)
            dataTrain = ut.Polinomial(dataTrain,k)


            pca.fit(dataTest)
            dataTest = pca.transform(dataTest)
            dataTest = pd.DataFrame(dataTest)
            dataTest = ut.Polinomial(dataTest,k)
            #size = dataTrain.shape[0]

            start_1 = timeit.default_timer()
            skf = StratifiedKFold(n_splits=int(10))
            skf.get_n_splits(dataTrain, yTrain)
            acc_kfold_Train = []
            f1_kfold_Train = []
            i = 0

            for train_index, test_index in skf.split(dataTrain, yTrain):
                        i = i+1
                        print('training fold ' + str(i))
                        X_train, X_test = dataTrain.iloc[train_index,:], dataTrain.iloc[test_index,:]
                        y_train, y_test = yTrain.iloc[train_index], yTrain.iloc[test_index]

                        basicLR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial',
                                                     C=Cvalue).fit(X_train, y_train)
                        predictions_1_Train = basicLR.predict(X_test)

                        accTrain = accuracy_score(y_test, predictions_1_Train, normalize=True)
                        f1Train = f1_score(y_test, predictions_1_Train, pos_label = 1)

                        if plot:
                            cm = confusion_matrix(y_test, predictions_1_Train)
                            ut.plot_confusion_matrix(cm, ['all', 'rrlyr'], type = 'train')
                        print('Accuracy train: ', accTrain)
                        #print('Accuracy f1 Train: ', f1Train)
                        basicLR_stats_acc.append(accTrain)
                        basicLR_stats_f1.append(f1Train)
            accTrain = np.mean(basicLR_stats_acc)
            f1Train =np.mean(basicLR_stats_f1)
            print('----------------------------------------------------------')
            print('Summary Training')
            print('Mean Accuracy train: ', accTrain)
            print('Mean f1 Train: ', f1Train)
            stop_1 = timeit.default_timer()
            time_CV = stop_1 - start_1
            basicLR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial', C=Cvalue).fit(dataTrain, yTrain)
            predictions_1_Test = basicLR.predict(dataTest)
            print(predictions_1_Test.shape)
            print(yTestHO.shape)
            print('----------------------------------------------------------')
            print('Summary Testing')
            accTest = accuracy_score(yTestHO, predictions_1_Test, normalize=True)
            f1Test = f1_score(yTestHO, predictions_1_Test,pos_label = 1)
            if plot: 
                cm = confusion_matrix(yTestHO, predictions_1_Test)
                ut.plot_confusion_matrix(cm, ['all', 'rrlyr'], type = 'train')

            print(' Accuracy test: ', accTest)
            print(' f1 test: ', f1Test)
            print('----------------------------------------------------------')

            res.append([k, components, size,  accTrain, accTest, f1Train, f1Test])
            pd.DataFrame(res).to_csv(str(number)+'-BasicCV0707C'+str(Cvalue)+'.csv')

kernel 1
components 2
71
training fold 1
Accuracy train:  0.9
training fold 2
Accuracy train:  0.92
training fold 3
Accuracy train:  0.89
training fold 4
Accuracy train:  0.86
training fold 5
Accuracy train:  0.87
training fold 6
Accuracy train:  0.86
training fold 7
Accuracy train:  0.88
training fold 8
Accuracy train:  0.89
training fold 9
Accuracy train:  0.85
training fold 10
Accuracy train:  0.85
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.877
Mean f1 Train:  0.8782998697207441
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.6805520999940762
 f1 test:  0.38332227889211634
----------------------------------------------------------
components 2
71
training fold 1
Accuracy train:  0.915
training fold 2
Accuracy train:  0.88
training fold 3
Accuracy train:  0.87
training fold 4
Accuracy train:  0.905
training fold 5
Accuracy train:  0.84
training fold 6
Accuracy trai

71
training fold 1
Accuracy train:  0.955
training fold 2
Accuracy train:  0.9475
training fold 3
Accuracy train:  0.945
training fold 4
Accuracy train:  0.9725
training fold 5
Accuracy train:  0.945
training fold 6
Accuracy train:  0.9325
training fold 7
Accuracy train:  0.9325
training fold 8
Accuracy train:  0.9275
training fold 9
Accuracy train:  0.965
training fold 10
Accuracy train:  0.9375
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9266041666666667
Mean f1 Train:  0.9283776986927609
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7973105858657662
 f1 test:  0.5035115212722735
----------------------------------------------------------
components 10
71
training fold 1
Accuracy train:  0.93
training fold 2
Accuracy train:  0.96
training fold 3
Accuracy train:  0.94
training fold 4
Accuracy train:  0.95
training fold 5
Accuracy train:  0.9
training fold 6
Accuracy

71
training fold 1
Accuracy train:  0.97
training fold 2
Accuracy train:  0.965
training fold 3
Accuracy train:  0.93
training fold 4
Accuracy train:  0.97
training fold 5
Accuracy train:  0.93
training fold 6
Accuracy train:  0.945
training fold 7
Accuracy train:  0.945
training fold 8
Accuracy train:  0.96
training fold 9
Accuracy train:  0.95
training fold 10
Accuracy train:  0.955
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9319347826086956
Mean f1 Train:  0.9340242986258267
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7048989988744743
 f1 test:  0.4109912977676883
----------------------------------------------------------
components 4
71
training fold 1
Accuracy train:  0.9675
training fold 2
Accuracy train:  0.9575
training fold 3
Accuracy train:  0.9475
training fold 4
Accuracy train:  0.9625
training fold 5
Accuracy train:  0.9525
training fold 6
Accuracy t

 Accuracy test:  0.7543628931935312
 f1 test:  0.45649723438278234
----------------------------------------------------------
components 12
71
training fold 1
Accuracy train:  0.94
training fold 2
Accuracy train:  0.97
training fold 3
Accuracy train:  0.97
training fold 4
Accuracy train:  0.97
training fold 5
Accuracy train:  0.93
training fold 6
Accuracy train:  0.99
training fold 7
Accuracy train:  0.96
training fold 8
Accuracy train:  0.96
training fold 9
Accuracy train:  0.94
training fold 10
Accuracy train:  0.94
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.940595588235294
Mean f1 Train:  0.9424295487517136
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.698998874474261
 f1 test:  0.3970190345089476
----------------------------------------------------------
components 12
71
training fold 1
Accuracy train:  0.955
training fold 2
Accuracy train:  0.97
training fold 

 Accuracy test:  0.6761329305135951
 f1 test:  0.3834912043301759
----------------------------------------------------------
components 6
71
training fold 1
Accuracy train:  0.9725
training fold 2
Accuracy train:  0.965
training fold 3
Accuracy train:  0.94
training fold 4
Accuracy train:  0.9725
training fold 5
Accuracy train:  0.9575
training fold 6
Accuracy train:  0.9525
training fold 7
Accuracy train:  0.94
training fold 8
Accuracy train:  0.96
training fold 9
Accuracy train:  0.9825
training fold 10
Accuracy train:  0.965
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9414555555555555
Mean f1 Train:  0.9434198719373386
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7473017001362479
 f1 test:  0.44973039911251
----------------------------------------------------------
components 8
71
training fold 1
Accuracy train:  0.93
training fold 2
Accuracy train:  0.98
traini

# Basic Logistic Regresion (Polynomial)

In [6]:
plot = False
basicLR_stats_acc = []
basicLR_stats_f1 = []
res = []
for k in [1, 2, 3]:
    print('kernel '+str(k))
    for components in [2, 4, 6, 8, 10, 12]:
        for size in[1000, 2000, 4000]:
            print('components '+str(components))
            dataTrain = pd.read_csv(fileTrain)
            dataTest = pd.read_csv(fileTest)
            try:
                dataTrain = dataTrain.loc[:, ~dataTrain.columns.str.contains('^Unnamed')]
                dataTrain =  dataTrain.drop(['Pred', 'Pred2', 'h', 'e', 'u','ID'], axis = 1)
                dataTrain = dataTrain.loc[:, (dataTrain != dataTrain.iloc[0]).any()]
                dataTest = dataTest[list(dataTrain.columns)]

            except:
                print('---')
            dataTrain = ut.downSampling(dataTrain)
            try:
                dataTrain = dataTrain.sample(size, random_state = 0)
            except:
                print('sample bigger that population')
            yTrain = 1*(dataTrain['label'] == 'ClassA')
            yTestHO = 1*(dataTest['label'] == 'ClassA')
            del dataTrain['label']
            del dataTest['label']

            names = dataTrain.columns
            scaler = preprocessing.StandardScaler()
            dataTrain = scaler.fit_transform(dataTrain)
            dataTrain = pd.DataFrame(dataTrain, columns=names)

            dataTest = scaler.fit_transform(dataTest)
            dataTest = pd.DataFrame(dataTest, columns=names)


            pca = PCA(n_components=components)
            pca.fit(dataTrain)
            dataTrain = pca.transform(dataTrain)
            dataTrain = pd.DataFrame(dataTrain)
            dataTrain = ut.Polinomial(dataTrain,k)


            pca.fit(dataTest)
            dataTest = pca.transform(dataTest)
            dataTest = pd.DataFrame(dataTest)
            dataTest = ut.Polinomial(dataTest,k)
            #size = dataTrain.shape[0]

            start_1 = timeit.default_timer()
            skf = StratifiedKFold(n_splits=int(10))
            skf.get_n_splits(dataTrain, yTrain)
            acc_kfold_Train = []
            f1_kfold_Train = []
            i = 0

            for train_index, test_index in skf.split(dataTrain, yTrain):
                        i = i+1
                        print('training fold ' + str(i))
                        X_train, X_test = dataTrain.iloc[train_index,:], dataTrain.iloc[test_index,:]
                        y_train, y_test = yTrain.iloc[train_index], yTrain.iloc[test_index]

                        basicLR = LogisticRegression(penalty='none', solver ='saga').fit(X_train, y_train)
                        predictions_1_Train = basicLR.predict(X_test)

                        accTrain = accuracy_score(y_test, predictions_1_Train, normalize=True)
                        f1Train = f1_score(y_test, predictions_1_Train, pos_label = 1)

                        if plot:
                            cm = confusion_matrix(y_test, predictions_1_Train)
                            ut.plot_confusion_matrix(cm, ['all', 'rrlyr'], type = 'train')
                        print('Accuracy train: ', accTrain)
                        #print('Accuracy f1 Train: ', f1Train)
                        basicLR_stats_acc.append(accTrain)
                        basicLR_stats_f1.append(f1Train)
            accTrain = np.mean(basicLR_stats_acc)
            f1Train =np.mean(basicLR_stats_f1)
            print('----------------------------------------------------------')
            print('Summary Training')
            print('Mean Accuracy train: ', accTrain)
            print('Mean f1 Train: ', f1Train)
            stop_1 = timeit.default_timer()
            time_CV = stop_1 - start_1
            basicLR = LogisticRegression(penalty='none', solver ='saga').fit(dataTrain, yTrain)
            predictions_1_Test = basicLR.predict(dataTest)
            print(predictions_1_Test.shape)
            print(yTestHO.shape)
            print('----------------------------------------------------------')
            print('Summary Testing')
            accTest = accuracy_score(yTestHO, predictions_1_Test, normalize=True)
            f1Test = f1_score(yTestHO, predictions_1_Test,pos_label = 1)
            if plot: 
                cm = confusion_matrix(yTestHO, predictions_1_Test)
                ut.plot_confusion_matrix(cm, ['all', 'rrlyr'], type = 'train')

            print(' Accuracy test: ', accTest)
            print(' f1 test: ', f1Test)
            print('----------------------------------------------------------')

            res.append([k, components, size,  accTrain, accTest, f1Train, f1Test])
            pd.DataFrame(res).to_csv('BasicCVAllDatawhitoutpenalty-0707'+str(number)+'.csv')

kernel 1
components 2
71
training fold 1
Accuracy train:  0.9
training fold 2
Accuracy train:  0.92
training fold 3
Accuracy train:  0.89
training fold 4
Accuracy train:  0.86
training fold 5
Accuracy train:  0.87
training fold 6
Accuracy train:  0.86
training fold 7
Accuracy train:  0.88
training fold 8
Accuracy train:  0.89
training fold 9
Accuracy train:  0.85
training fold 10
Accuracy train:  0.85
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.877
Mean f1 Train:  0.8782998697207441
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.6805402523547183
 f1 test:  0.3833135120300064
----------------------------------------------------------
components 2
71
training fold 1
Accuracy train:  0.915
training fold 2
Accuracy train:  0.88
training fold 3
Accuracy train:  0.87
training fold 4
Accuracy train:  0.905
training fold 5
Accuracy train:  0.84
training fold 6
Accuracy train

71
training fold 1
Accuracy train:  0.955
training fold 2
Accuracy train:  0.95
training fold 3
Accuracy train:  0.9475
training fold 4
Accuracy train:  0.975
training fold 5
Accuracy train:  0.9425
training fold 6
Accuracy train:  0.9325
training fold 7
Accuracy train:  0.9375
training fold 8
Accuracy train:  0.9275
training fold 9
Accuracy train:  0.9675
training fold 10
Accuracy train:  0.94
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9265624999999998
Mean f1 Train:  0.9283530182686128
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7963035365203484
 f1 test:  0.502560541619651
----------------------------------------------------------
components 10
71
training fold 1
Accuracy train:  0.93
training fold 2
Accuracy train:  0.97
training fold 3
Accuracy train:  0.94
training fold 4
Accuracy train:  0.95
training fold 5
Accuracy train:  0.9
training fold 6
Accuracy tr

71
training fold 1
Accuracy train:  0.955
training fold 2
Accuracy train:  0.94
training fold 3
Accuracy train:  0.92
training fold 4
Accuracy train:  0.96
training fold 5
Accuracy train:  0.905
training fold 6
Accuracy train:  0.94
training fold 7
Accuracy train:  0.935
training fold 8
Accuracy train:  0.935
training fold 9
Accuracy train:  0.94
training fold 10
Accuracy train:  0.955
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9302608695652173
Mean f1 Train:  0.9319579808386647
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7521829275516854
 f1 test:  0.45319321360416176
----------------------------------------------------------
components 4
71
training fold 1
Accuracy train:  0.9225
training fold 2
Accuracy train:  0.885
training fold 3
Accuracy train:  0.865
training fold 4
Accuracy train:  0.925
training fold 5
Accuracy train:  0.895
training fold 6
Accuracy tra

71
training fold 1
Accuracy train:  0.95
training fold 2
Accuracy train:  0.96
training fold 3
Accuracy train:  0.96
training fold 4
Accuracy train:  0.94
training fold 5
Accuracy train:  0.9
training fold 6
Accuracy train:  0.97
training fold 7
Accuracy train:  0.94
training fold 8
Accuracy train:  0.93
training fold 9
Accuracy train:  0.92
training fold 10
Accuracy train:  0.93
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9323970588235294
Mean f1 Train:  0.9335387121817741
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7167466382323322
 f1 test:  0.41367471061408667
----------------------------------------------------------
components 12
71
training fold 1
Accuracy train:  0.955
training fold 2
Accuracy train:  0.925
training fold 3
Accuracy train:  0.92
training fold 4
Accuracy train:  0.955
training fold 5
Accuracy train:  0.94
training fold 6
Accuracy train:  0.9

(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.67372785972395
 f1 test:  0.36870458244504045
----------------------------------------------------------
components 6
71
training fold 1
Accuracy train:  0.845
training fold 2
Accuracy train:  0.8675
training fold 3
Accuracy train:  0.835
training fold 4
Accuracy train:  0.81
training fold 5
Accuracy train:  0.7975
training fold 6
Accuracy train:  0.8225
training fold 7
Accuracy train:  0.78
training fold 8
Accuracy train:  0.8475
training fold 9
Accuracy train:  0.8525
training fold 10
Accuracy train:  0.8175
----------------------------------------------------------
Summary Training
Mean Accuracy train:  0.9158000000000001
Mean f1 Train:  0.91651743320429
(84405,)
(84405,)
----------------------------------------------------------
Summary Testing
 Accuracy test:  0.7130975653101119
 f1 test:  0.4138264910921766
----------------------------------------------------------
comp